In [ ]:
import pandas as pd
import requests
from IPython.display import clear_output
import time

In [ ]:
final_df = pd.read_csv('데마프로젝트최종데이터.csv',index_col =0)
final_df.head()

In [ ]:
for i in [1,2,3,4,5,6,7,8,9,10,17,18,19,20,21]:
    final_df[final_df.columns[i]] = final_df[final_df.columns[i]].astype('str') 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz


In [ ]:
final_df.dtypes

In [ ]:
class_df = final_df['win']

In [ ]:
# 인게임 데이터
# 인게임 요소 기준 FI(첫 억제기 파괴) 제외
train_pre = final_df.loc[:,['FB','FT','FD','FJ' ]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_pre, class_df, stratify = class_df, random_state=42,train_size = 0.25)

In [ ]:
#decision Tree
DT_accuracy_list =[]
DT_model_list = []
for depth in range(1,8):
    tree = DecisionTreeClassifier(max_depth = depth, random_state=123)
    tree.fit(X_train, y_train)

    y_pred = tree.predict(X_test)
    accuracy = accuracy_score(y_pred, y_test)
    DT_accuracy_list.append(accuracy)
    DT_model_list.append(tree)


best_index = DT_accuracy_list.index(max(DT_accuracy_list))+1
print("테스트 정확도: {:.3f}".format(max(DT_accuracy_list)))
print('최대 깊이 : {:d}'.format(best_index))
best_DT = DT_model_list[best_index-1]

In [ ]:
best_DT.feature_importances_

In [ ]:
importance = best_DT.feature_importances_
importance_list = list(zip(X_train.columns, importance))

sorted(importance_list, key =lambda x:x[1], reverse=True)

In [ ]:
# random forest
RF_accuracy_list = []
RF_model_list =[]
for i in range(10):
    clf_rf = RandomForestClassifier()
    clf_rf.fit(X_train, y_train)
    y_pred = clf_rf.predict(X_test)
    accuracy = accuracy_score(y_pred, y_test)
    RF_accuracy_list.append(accuracy)
    RF_model_list.append(clf_rf)
    
print("테스트 정확도: {:.3f}".format(max(RF_accuracy_list)))
print(f'가장 성능이 좋은 모델: {(DT_accuracy_list.index(max(DT_accuracy_list)))+1}번째 모델')
best_RF = RF_model_list[(DT_accuracy_list.index(max(DT_accuracy_list)))]

In [ ]:
best_RF.feature_importances_

In [ ]:
importance2 = best_RF.feature_importances_
importance_list2 = list(zip(X_train.columns, importance2))

sorted(importance_list2, key =lambda x:x[1], reverse=True)